# Umgebungsvariablen

In [15]:
train_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/train.csv'''
feature_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/'''

# Import

In [93]:
import time

import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
import librosa

from scipy.stats import kurtosis
from scipy.stats import skew

from scipy import signal

import warnings
warnings.filterwarnings('ignore')

pd.options.display.precision = 10

# Trainingsdaten laden

In [17]:
train_data = pd.read_csv(train_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

## Low-pass (noise-reducing) filter

In [329]:
def downsample(data, downsample_rate):
    trunc = int((data.shape[0] // downsample_rate) * downsample_rate)
    data = data[:trunc, :]
    x = signal.decimate(data[:, 0].ravel(), downsample_rate)
    y_ = data[:, 1].reshape((-1, downsample_rate)).T
    y = y_.mean(axis=0)
    return x, y

In [94]:
#The scipy.signal.decimate function applies a low-pass (noise-reducing) filter, and samples the data every n-th value based on the downsample rate
dsamprate = 4
train_data_lowpassfilter = signal.decimate(train_data.acoustic_data, dsamprate)
train_data_lowpassfilter['time_to_failure'] = train_data.time_to_failure[::dsamprate]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [319]:
time_to_failure_list = list(train_data.time_to_failure)[::dsamprate]

# Feature Extraction

Die Testsegmente bestehen jeweils aus 150.000 Messpunkte. Aus diesem Grund setzten wir die Fenstergrösse auf 150000 Messpunkte.

## Feature Extraction Function

Extracting features
We will extract

 - statistical moments
 - root-mean-square
 - Zero Crossing Rate
 - Spectral Centroid
 - Chroma Frequencies
 - Spectral Rolloff
 - Mel-Frequency Cepstral Coefficients

In [307]:
def generate_featureRow(acoustic_data, time_to_failure, sample_rate):
    
    strain_feature = []
    strain_feature.append(acoustic_data.mean())
    strain_feature.append(acoustic_data.std())
    strain_feature.append(acoustic_data.kurtosis())
    strain_feature.append(acoustic_data.skew())
    strain_feature.append(acoustic_data.min())
    strain_feature.append(acoustic_data.max())
    strain_feature.append(acoustic_data.quantile(0.01, interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.05,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.25,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.75,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.95,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.99,interpolation='midpoint'))
    strain_feature.append(acoustic_data.abs().max())
    strain_feature.append(acoustic_data.abs().mean())
    strain_feature.append(acoustic_data.abs().std())
    
    #Compute root-mean-square (RMS) value for each frame
    rms = librosa.feature.rms(y=np.array(acoustic_data,dtype=np.float64),frame_length=len(acoustic_data))
    rms = pd.Series(rms[0,:])
    strain_feature.append(rms.mean())
    strain_feature.append(rms.std())
    strain_feature.append(rms.kurtosis())
    strain_feature.append(rms.skew())
    strain_feature.append(rms.min())
    strain_feature.append(rms.max())
    strain_feature.append(rms.quantile(0.01, interpolation='midpoint'))
    strain_feature.append(rms.quantile(0.05,interpolation='midpoint'))
    strain_feature.append(rms.quantile(0.25,interpolation='midpoint'))
    strain_feature.append(rms.quantile(0.75,interpolation='midpoint'))
    strain_feature.append(rms.quantile(0.95,interpolation='midpoint'))
    strain_feature.append(rms.quantile(0.99,interpolation='midpoint'))
    strain_feature.append(rms.abs().max())
    strain_feature.append(rms.abs().mean())
    strain_feature.append(rms.abs().std())
    
    #Zero Crossing Rate
    '''The zero crossing rate is the rate of sign-changes along a signal, i.e., 
    the rate at which the signal changes from positive to negative or back.'''
    zero_crossings = librosa.zero_crossings(np.array(acoustic_data,dtype=np.float64), pad=False)
    strain_feature.append(sum(zero_crossings))
    
    #Spectral Centroid
    '''Each frame of a magnitude spectrogram is normalized and treated as a distribution
    over frequency bins, from which the mean (centroid) is extracted per frame.So spectral centroid for blues 
    song will lie somewhere near the middle of its spectrum while that for a metal song would be towards its end.'''
    spectral_centroids = librosa.feature.spectral_centroid(np.array(acoustic_data,dtype=np.float64), sr=sample_rate)[0]
    spectral_centroids = pd.Series(spectral_centroids)
    strain_feature.append(spectral_centroids.mean())
    strain_feature.append(spectral_centroids.std())
    strain_feature.append(spectral_centroids.kurtosis())
    strain_feature.append(spectral_centroids.skew())
    strain_feature.append(spectral_centroids.min())
    strain_feature.append(spectral_centroids.max())
    strain_feature.append(spectral_centroids.quantile(0.01, interpolation='midpoint'))
    strain_feature.append(spectral_centroids.quantile(0.05,interpolation='midpoint'))
    strain_feature.append(spectral_centroids.quantile(0.25,interpolation='midpoint'))
    strain_feature.append(spectral_centroids.quantile(0.75,interpolation='midpoint'))
    strain_feature.append(spectral_centroids.quantile(0.95,interpolation='midpoint'))
    strain_feature.append(spectral_centroids.quantile(0.99,interpolation='midpoint'))
    strain_feature.append(spectral_centroids.abs().max())
    strain_feature.append(spectral_centroids.abs().mean())
    strain_feature.append(spectral_centroids.abs().std())
    
    
    
    #power spectral density
    f, Pxx_den = signal.welch(train_data_test, sample_rate)
    Pxx_den = Pxx_den[:,0]
    strain_feature.append(np.mean(Pxx_den))
    strain_feature.append(np.std(Pxx_den))
    strain_feature.append(kurtosis(Pxx_den))
    strain_feature.append(skew(Pxx_den))
    strain_feature.append(np.min(Pxx_den))
    strain_feature.append(np.max(Pxx_den))
    strain_feature.append(np.quantile(Pxx_den,0.01, interpolation='midpoint'))
    strain_feature.append(np.quantile(Pxx_den,0.05,interpolation='midpoint'))
    strain_feature.append(np.quantile(Pxx_den,0.25,interpolation='midpoint'))
    strain_feature.append(np.quantile(Pxx_den,0.75,interpolation='midpoint'))
    strain_feature.append(np.quantile(Pxx_den,0.95,interpolation='midpoint'))
    strain_feature.append(np.quantile(Pxx_den,0.99,interpolation='midpoint'))
   
    #Chroma Frequencies
    '''Chroma features are an interesting and powerful representation for music audio in which the entire 
    spectrum is projected onto 12 bins representing the 12 distinct semitones (or chroma) of the musical octave.'''
    chroma_stft = librosa.feature.chroma_stft(y=np.array(acoustic_data,dtype=np.float64), sr=sample_rate)
    strain_feature.append(np.mean(chroma_stft))
    strain_feature.append(np.std(chroma_stft))
    strain_feature.append(kurtosis(chroma_stft[0]))
    strain_feature.append(skew(chroma_stft[0]))
    strain_feature.append(np.min(chroma_stft))
    strain_feature.append(np.max(chroma_stft))
    strain_feature.append(np.quantile(chroma_stft,0.01, interpolation='midpoint'))
    strain_feature.append(np.quantile(chroma_stft,0.05,interpolation='midpoint'))
    strain_feature.append(np.quantile(chroma_stft,0.25,interpolation='midpoint'))
    strain_feature.append(np.quantile(chroma_stft,0.75,interpolation='midpoint'))
    strain_feature.append(np.quantile(chroma_stft,0.95,interpolation='midpoint'))
    strain_feature.append(np.quantile(chroma_stft,0.99,interpolation='midpoint'))
    
    #Spectral Rolloff
    '''It is a measure of the shape of the signal. 
    It represents the frequency below which a specified percentage of the total spectral energy, e.g. 85%, lies.'''
    rolloff = librosa.feature.spectral_rolloff(y=np.array(acoustic_data,dtype=np.float64), sr=sample_rate)[0]
    strain_feature.append(np.mean(rolloff))
    strain_feature.append(np.std(rolloff))
    strain_feature.append(kurtosis(rolloff))
    strain_feature.append(skew(rolloff))
    strain_feature.append(np.min(rolloff))
    strain_feature.append(np.max(rolloff))
    strain_feature.append(np.quantile(rolloff,0.01, interpolation='midpoint'))
    strain_feature.append(np.quantile(rolloff,0.05,interpolation='midpoint'))
    strain_feature.append(np.quantile(rolloff,0.25,interpolation='midpoint'))
    strain_feature.append(np.quantile(rolloff,0.75,interpolation='midpoint'))
    strain_feature.append(np.quantile(rolloff,0.95,interpolation='midpoint'))
    strain_feature.append(np.quantile(rolloff,0.99,interpolation='midpoint'))
    
    #spectral_bandwidth
    spec_bw = librosa.feature.spectral_bandwidth(y=np.array(acoustic_data,dtype=np.float64), sr=sample_rate)
    spec_bw = spec_bw[0,:]
    strain_feature.append(np.mean(spec_bw))
    strain_feature.append(np.std(spec_bw))
    strain_feature.append(kurtosis(spec_bw))
    strain_feature.append(skew(spec_bw))
    strain_feature.append(np.min(spec_bw))
    strain_feature.append(np.max(spec_bw))
    strain_feature.append(np.quantile(spec_bw,0.01, interpolation='midpoint'))
    strain_feature.append(np.quantile(spec_bw,0.05,interpolation='midpoint'))
    strain_feature.append(np.quantile(spec_bw,0.25,interpolation='midpoint'))
    strain_feature.append(np.quantile(spec_bw,0.75,interpolation='midpoint'))
    strain_feature.append(np.quantile(spec_bw,0.95,interpolation='midpoint'))
    strain_feature.append(np.quantile(spec_bw,0.99,interpolation='midpoint'))
    
    #Time to failure
    strain_feature.append(time_to_failure.values[-1])
    
    return strain_feature

Window wird durch das Trainingssignal geschoben mit einer Überlappung von step

In [308]:
def generate_Features(train,step,sample_rate):
    
    window_size=150000
    
    init_idx = 0
    features = []

    while (init_idx + window_size < len(train)):
        window = train.iloc[init_idx:init_idx + window_size]
        features.append(generate_featureRow(window.acoustic_data,window.time_to_failure,sample_rate))
        init_idx += step
   
    return pd.DataFrame(features,dtype=np.float64,columns =  ['mean','std','kurt','skew','min','max','1%q','5%q','25%q','75%q','95%q','99%q','absMax','absMean','absStd',
                                                            'rms_mean','rms_std','rms_kurt','rms_skew','rms_min','rms_max','rms_1%q','rms_5%q','rms_25%q','rms_75%q','rms_95%q','rms_99%q','rms_absMax','rms_absMean','rms_absStd','zero_crossings',
                                                             'spec_cent_mean','spec_cent_std','spec_cent_kurt','spec_cent_skew','spec_cent_min','spec_cent_max','spec_cent_1%q','spec_cent_5%q','spec_cent_25%q','spec_cent_75%q','spec_cent_95%q','spec_cent_99%q','spec_cent_absMax','spec_cent_absMean','spec_cent_absStd',
                                                             'psd_mean','psd_std','psd_kurt','psd_skew','psd_min','psd_max','psd_1%q','psd_5%q','psd_25%q','psd_75%q','psd_95%q','psd_99%q',
                                                             'chroma_stft_mean','chroma_stft_std','chroma_stft_kurt','chroma_stft_skew','chroma_stft_min','chroma_stft_max','chroma_stft_1%q','chroma_stft_5%q','chroma_stft_25%q','chroma_stft_75%q','chroma_stft_95%q','chroma_stft_99%q',
                                                             'rolloff_mean','rolloff_std','rolloff_kurt','rolloff_skew','rolloff_min','rolloff_max','rolloff_1%q','rolloff_5%q','rolloff_25%q','rolloff_75%q','rolloff_95%q','rolloff_99%q',
                                                             'spec_bw_mean','spec_bw_std','spec_bw_kurt','spec_bw_skew','spec_bw_min','spec_bw_max','spec_bw_1%q','spec_bw_5%q','spec_bw_25%q','spec_bw_75%q','spec_bw_95%q','spec_bw_99%q',
                                                        
                                                             'time_to_failure'])

In [309]:
# https://www.kaggle.com/flaport/linear-regression-on-180-features/notebook

## Feature extrahieren

### Testhalber eine Reihe extrahieren

In [310]:
step = 150000
train_data_test = train_data.iloc[0:151000,:] 

In [311]:
sample_rate = 4000000
features = generate_Features(train_data_test,step,sample_rate)

In [318]:
features.iloc[:,81:102]

,rolloff_99%q,spec_bw_mean,spec_bw_std,spec_bw_kurt,spec_bw_skew,spec_bw_min,spec_bw_max,spec_bw_1%q,spec_bw_5%q,spec_bw_25%q,spec_bw_75%q,spec_bw_95%q,spec_bw_99%q,time_to_failure
0,1511718.75,563022.3012635449,26883.0330362413,12.7760446198,-2.9176274163,381349.1865900679,593848.0949585186,464802.9437660334,513229.8197951604,557429.6937546992,578910.9580815419,586597.8954874629,589247.3423812583,1.4307971859


### Überlappung definieren

In [ ]:
sample_rate = 4000000
step = 15000

## Feature extrahieren

In [ ]:
features = generate_Features(train_data,step,sample_rate)

### Generierte Feature sichern

In [12]:
features = pd.DataFrame()

In [23]:
# saving the dataframe
timestamp = time.time()
features.to_csv(feature_data_path+'earthquakeFeatures_'+str(timestamp)+'_.csv', header=True, index=True)

# Feature laden

In [ ]:
feature_data_path = ''

In [ ]:
features = pd.read_csv(feature_data_path,dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

## New Feature

 - Sehr interessant für neue Feature: https://towardsdatascience.com/music-genre-classification-with-python-c714d032f0d8 
 - spectral_features https://musicinformationretrieval.com/spectral_features.html
 
 

- Signal: phase, frequency, angular frequency, wavelength and a period. --> new features for mean / std / skew / kurt / percentile https://pythontic.com/visualization/signals/magnitude%20spectrum

 - RMS: root mean square of a signal (Effektivwert): https://de.wikipedia.org/wiki/Effektivwert
 - https://librosa.github.io/librosa/feature.html (RMS/etc..) ()